# 1. Crawler

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [1]:
import argparse
import os
import requests
import urllib

def wget(address, filename):
    # allow redirects - in case file is relocated
    resp = requests.get(address, allow_redirects=True)

    # check if request got satisfied
    if not resp.ok:
        print(resp.status_code, resp.reason, 'for', address)
        return
    else:
        # just to be cool and print something
        print(f"Request status: {resp.status_code}.")
    
    # try to extract filename from address
    url = urllib.parse.urlparse(address)
    local_path = url.path
    filename = os.path.basename(local_path)
    if filename == "":
        raise NameError(f"Filename neither given, nor found for {address}")

    extension = os.path.splitext(filename)[-1]

    # there are 4 things required to uniquely indetify file location in global network:
    # 1. Hostname 2. Port 3. Local path on "Hostname:Port" 4. Query
    # note: despite server IS able to return some file without priding its local path
    # we assume that its location is required to identify file type
    new_name = str(hash("<|>".join([url.hostname, str(url.port), local_path, url.query])))

    if os.path.exists(new_name + extension):
        raise OSError(f"File {new_name + extension} already exists")
    
    with open(new_name + extension, 'wb') as f:
        f.write(resp.content)
        print(f"File saved as {new_name + extension}")


# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description='download file.')
#     parser.add_argument("-O", type=str, default=None, dest='filename', help="output file name. Default -- taken from resource")
#     parser.add_argument("address", type=str, default=None, help="Provide URL here")
#     args = parser.parse_args()
#     wget(args.address, args.filename)

### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [2]:
import requests
import urllib
import traceback
import socket

class Document:
    
    def __init__(self, url):
        self.url = url
        self.content = None
        
    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def unique_name(self) -> str:
        url = urllib.parse.urlparse(self.url)
        local_path = url.path
        extension = ""
        # try to extract filename from url
        if os.path.basename(local_path) != "":
            extension = os.path.splitext(os.path.basename(local_path))[1]

        # there are 4 things required to uniquely indetify file location in global network:
        # 1. Hostname 2. Port 3. Local path on "Hostname:Port" 4. Query
        # note: despite server IS able to return some file without priding its local path
        # we assume that its location is required to identify file type
        host = "" if url.hostname is None else url.hostname
        new_name = str(hash("<|>".join([host, str(url.port), local_path, url.query])))
        return new_name + extension

    def download(self) -> bool:
        try:
            response = requests.get(self.url, allow_redirects=True)
            self.content = response.content
            return response.ok
        except TimeoutError:
            print("REQUEST ERROR: Timeout")
        except requests.exceptions.TooManyRedirects:
            print("REQUEST ERROR: Too many redirects")
        except requests.exceptions.InvalidSchema:
            print(f"REQUEST ERROR: {self.url} is not an html address")
        except requests.exceptions.RequestException as e:
            print(f"REQUEST ERROR: (see below)")
            print(traceback.format_exc(e))
        except ConnectionError as e:
            print(f"CONNECTION ERROR")
            print(traceback.format_exc(e))
        return False
    
    def persist(self) -> bool:
        if self.content is None:
            return False
        filename = self.unique_name()
        if os.path.exists(filename):
            return False
        with open(filename, 'wb') as f:
            f.write(self.content)
        return True
            
    def load(self):
        filename = self.unique_name()
        if not os.path.exists(filename):
            return False
        with open(filename, 'rb') as f:
            self.content = f.read()
        return True

### 1.1.1. Tests ###

In [3]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

## 1.2. [M][15] Account the caching policy

Sometimes remote documents (especially when we speak about static content like `js` or `gif`) can swear that they will not change for some time. This is done by setting [Cache-Control response header](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Cache-Control).

In [4]:
import requests
requests.get('https://polyfill.io/v3/polyfill.min.js').headers['Cache-Control']

'public, s-maxage=31536000, max-age=604800, stale-while-revalidate=604800, stale-if-error=604800'

Please study the documentation and implement a descendant to a `Document` class, which will refresh the document in case of expired cache even if the file is already on the hard drive.

In [5]:
class CachedDocument(Document):
    
    # TODO your code here
    pass    

### 1.2.1. Tests

Add logging in your code and show that your code behaves differently for documents with different caching policy.

In [6]:
import time

doc = CachedDocument('https://polyfill.io/v3/polyfill.min.js')
doc.get()
time.sleep(2)
doc.get()
time.sleep(2)
doc.get()

doc = CachedDocument('https://yandex.ru/')
doc.get()
time.sleep(2)
doc.get()
time.sleep(2)
doc.get()

## 1.3. [10] Parse HTML ##
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
- `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
- `self.images` list of images met in a document. Again, links can be relative to current page.
- `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [7]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib


class HtmlDocument(Document):
    
    def parse(self):
        # do not parse content files 
        if any(map(lambda ext : self.unique_name().endswith(ext), ['.pdf', '.mp3', '.avi', '.mp4', '.txt'])):
            self.text = ""
            self.anchors = ""
            self.images = ""
            return

        soup = BeautifulSoup(self.content)

        # combine all consecutive space characters into one
        self.text = "\n".join([s.strip() for s in soup.get_text().split("\n") if s.strip()])

        self.images = [urllib.parse.urljoin(self.url, img.get("src")) for img in soup.findAll("img")]
        
        self.anchors = []
        for link in soup.findAll("a"):
            if link.get("href"):
                self.anchors.append(("" if link.string is None else link.string.strip(), urllib.parse.urljoin(self.url, link.get("href"))))

### 1.3.1. Tests ###

In [8]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

## 1.4. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose). 

**Criteria of success**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained inside `<body>` tag only.

In [9]:
from collections import Counter

class HtmlDocumentTextData:
    
    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()
    
    def get_sentences(self):
        return [s.strip() for s in self.doc.text.replace("\n", ".").split(".")]
    
    def get_word_stats(self):
        words = []
        for s in self.get_sentences():
            words += ["".join(filter(str.isalpha, w.lower())) for w in s.split() if "".join(filter(str.isalpha, w)) != ""]
        return Counter(words)

### 1.4.1. Tests ###

In [10]:
doc = HtmlDocumentTextData("https://innopolis.university/")

print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

[('и', 60), ('в', 35), ('иннополис', 22), ('по', 16), ('университет', 13), ('на', 11), ('области', 10), ('лаборатория', 10), ('университета', 9), ('центр', 9)]


## 1.5. [M][35] Languages
Maybe you heard, that there are multiple languages in the world. European languages, like Russian and English, use similar puctuation, but even in this family there is ¡Spanish!

Other languages can use different punctiation rules, like **Arabic or [Thai](http://www.thai-language.com/ref/breaking-words)**.

Your task is to support (at least) three languages (English, Arabic, and Thai) tokenization in your `HtmlDocumentTextData` class descendant.

What should you do:
1. Use any language dection techniques, e.g. [langdetect](https://pypi.org/project/langdetect/).
2. Use language-specific tokenization tools, e.g. for [Thai](https://pythainlp.github.io/tutorials/notebooks/pythainlp_get_started.html#Tokenization-and-Segmentation) and [Arabic](https://github.com/CAMeL-Lab/camel_tools).
3. Use these pages to test your code: [1](https://www.bangkokair.com/tha/baggage-allowance) and [2](https://alfajr-news.net/details/%D9%85%D8%B4%D8%B1%D9%88%D8%B9-%D8%AF%D9%8A%D9%85%D9%88%D9%82%D8%B1%D8%A7%D8%B7%D9%8A-%D9%81%D9%8A-%D8%A7%D9%84%D9%83%D9%88%D9%86%D8%BA%D8%B1%D8%B3-%D8%A7%D9%84%D8%A3%D9%85%D8%B1%D9%8A%D9%83%D9%8A-%D9%84%D9%85%D8%B9%D8%A7%D9%82%D8%A8%D8%A9-%D8%A8%D9%88%D8%AA%D9%8A%D9%86).

In [11]:
class MultilingualHtmlDocumentTextData(HtmlDocumentTextData):
    
    #TODO your code here
    pass

### 1.5.1. Tests

In [12]:
doc = MultilingualHtmlDocumentTextData("https://www.bangkokair.com/tha/baggage-allowance")
print(doc.get_word_stats().most_common(10))

doc = MultilingualHtmlDocumentTextData("https://alfajr-news.net/details/%D9%85%D8%B4%D8%B1%D9%88%D8%B9-%D8%AF%D9%8A%D9%85%D9%88%D9%82%D8%B1%D8%A7%D8%B7%D9%8A-%D9%81%D9%8A-%D8%A7%D9%84%D9%83%D9%88%D9%86%D8%BA%D8%B1%D8%B3-%D8%A7%D9%84%D8%A3%D9%85%D8%B1%D9%8A%D9%83%D9%8A-%D9%84%D9%85%D8%B9%D8%A7%D9%82")
print(doc.get_word_stats().most_common(10))

[('กโลกรม', 21), ('โซน', 11), ('x', 6), ('ภาษาไทย', 4), ('usd', 4), ('เขาสระบบ', 4), ('ตดตอเรา', 4), ('หรอ', 3), ('หนาหลก', 3), ('ขอมลการเดนทาง', 3)]
[('تعليق', 12), ('مشاهده', 10), ('و', 5), ('الفجر', 4), ('فن', 4), ('الإمارات', 4), ('alfajr', 3), ('بن', 3), ('أخبار', 3), ('أغسطس', 3)]


## 1.5. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [13]:
import traceback

class Crawler:
    def __init__(self):
        self.processed = set()
        self.queue = set()

    def crawl_generator(self, source, depth=1):
        #TODO return real crawling results. Don't forget to process failures
        self.queue.add((depth, source))

        while True:
            if len(self.queue) == 0:
                break
            current_depth, url = self.queue.pop()
            try:
                document = HtmlDocumentTextData(url)
            except FileNotFoundError:
                print(f"RESOURCE NOT FOUND: {url}")
            except Exception as e:
                print(f"FAIL ACCESSING {url}:")
                print(traceback.format_exc())
                continue
            self.processed.add(url)
            yield document

            if current_depth > 0:
                for link in document.doc.anchors:
                    if link[1] not in self.processed and link[1] not in self.queue:
                        self.queue.add((current_depth - 1, link[1]))

### 1.5. Tests ###

In [14]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'

https://innopolis.university/en/
316 distinct word(s) so far
https://www.instagram.com/innopolisu/
318 distinct word(s) so far
https://corporate.innopolis.university/en
505 distinct word(s) so far
http://www.campuslife.innopolis.ru/main
717 distinct word(s) so far
https://innopolis.university/en/lab-cyberphysical-systems/
807 distinct word(s) so far
https://innopolis.university/en/ido/
873 distinct word(s) so far
https://innopolis.university/lab-cyberphysical-systems/
1375 distinct word(s) so far
https://innopolis.university/en/team-structure/team-faculty/
1418 distinct word(s) so far
https://career.innopolis.university/en/job/
2034 distinct word(s) so far
RESOURCE NOT FOUND: https://university.innopolis.ru/en/cooperation/
https://career.innopolis.university/en/job/
2034 distinct word(s) so far
https://alumni.innopolis.university/
2347 distinct word(s) so far
https://innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
Skipping https://innopolis.university/publi

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://career.innopolis.university/public/files/career_personal_data.docx
87058 distinct word(s) so far
http://www.minsvyaz.ru/en/events/40758/
87079 distinct word(s) so far
https://t.me/joinchat/Q48uDKMDGcTJN1EH
87079 distinct word(s) so far
https://media.innopolis.university/en/events/olimpiada-innopolis-open-po-informatsionnoy-bezopasnosti/?TAGS=Blockchain
87079 distinct word(s) so far
https://media.innopolis.university/events/
87139 distinct word(s) so far
https://media.innopolis.university/en/events/olimpiada-innopolis-open-po-informatike/
87139 distinct word(s) so far
https://apply.innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
Skipping https://apply.innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
https://cs.gssi.it/devops2020/
87193 distinct word(s) so far
https://apply.innopolis.university/en/olympiad-bonus/
87206 distinct word(s) so far
https://innopolis.university/en/?special=Y
87206 distinct word(s) so far
http://ni

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://innopolis.university/filespublic/photo_2021-12-07_17-00-50.jpg
103130 distinct word(s) so far
https://www.youtube.com/
103131 distinct word(s) so far
http://www.google.ru/intl/en/policies/terms/
103132 distinct word(s) so far
http://scholar.google.ru/scholar_alerts?view_op=list_alerts&hl=en&oe=ASCII
103133 distinct word(s) so far
https://www.youtube.com/about/copyright/
103190 distinct word(s) so far
https://scholar.google.ru/scholar?oi=bibs&hl=en&oe=ASCII&cites=8355664610371031532
103203 distinct word(s) so far
http://www.campuslife.innopolis.ru/studentreps
103214 distinct word(s) so far
https://innopolis.university/en/centergis/
103225 distinct word(s) so far
https://accounts.google.com/Login?hl=en&continue=https://scholar.google.ru/schhp%3Fhl%3Den%26oe%3DASCII
103225 distinct word(s) so far
http://scholar.google.ru/citations?user=KLpMBj0AAAAJ&hl=en&oe=ASCII
103317 distinct word(s) so far
http://scholar.google.ru/citations?view_op=view_citation&hl=en&oe=ASCII&user=sVjTfqEAAAA